# Exploratory Data Analysis

# Questions to Answer

1. How have the adjusted close prices of selected stocks (AAPL, MSFT, and GOOGL) trended over time?
2. How do different sectors and industries perform in terms of returns and risk (volatility) over time?
3. Which sectors exhibit the highest volatility, and when?
4. What are the patterns in trading volume across different sectors?

In [1]:
# Loading Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Loading datasets
file_path = r'C:\Users\hp\.cache\kagglehub\datasets\andrewmvd\sp-500-stocks\versions\963\sp500_stocks.csv'

df_sp500 = pd.read_csv(file_path)
df_companies = pd.read_csv(r'C:\Users\hp\OneDrive\Desktop\Data_Analyst\S&P_500\sp500_data\sp500_companies.csv')

In [2]:
# Data Cleanup
# Cleaning df_sp500
# Convert Date to datetime format
df_sp500['Date'] = pd.to_datetime(df_sp500['Date'])

df_sp500['Symbol'] = df_sp500['Symbol'].astype('category')

df_cleaned = df_sp500.dropna()

In [3]:
# Merging Datasets
merged_df = pd.merge(df_cleaned, df_companies, on='Symbol', how='inner')

- Merging and Cleaning Datasets 

In [4]:
# Cleaning the merged datasets
# Remove Duplicates
merged_df.drop_duplicates(inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1773055 entries, 0 to 1773054
Data columns (total 23 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   Symbol               object        
 2   Adj Close            float64       
 3   Close                float64       
 4   High                 float64       
 5   Low                  float64       
 6   Open                 float64       
 7   Volume               float64       
 8   Exchange             object        
 9   Shortname            object        
 10  Longname             object        
 11  Sector               object        
 12  Industry             object        
 13  Currentprice         float64       
 14  Marketcap            int64         
 15  Ebitda               float64       
 16  Revenuegrowth        float64       
 17  City                 object        
 18  State                object        
 19  Country              

In [5]:
# Compute median employee count per sector ()
sector_medians = merged_df.groupby('Sector')['Fulltimeemployees'].transform('median')
merged_df['Fulltimeemployees'] = merged_df['Fulltimeemployees'].fillna(sector_medians)

In [6]:
# Ensure numerical columns are of the correct type
numerical_type_cols = ['Currentprice', 'Marketcap', 'Volume', 'Weight']
merged_df[numerical_type_cols] = merged_df[numerical_type_cols].apply(pd.to_numeric, errors='coerce')

In [7]:
# Step 1: Reset the index to work with 'Date' as a regular column
merged_df = merged_df.reset_index()

# Step 2: Calculate Weekly Returns per Symbol
# Group by 'Symbol', set 'Date' as index, and resample weekly
merged_df['Weekly_Return'] = (
    merged_df.groupby('Symbol')['Adj Close']
    .apply(lambda x: x.resample('W-MON', on='Date').ffill().pct_change())
)

# Step 3: Calculate 30-day and 90-day Rolling Volatility for Daily Returns
merged_df['Daily_Return'] = merged_df.groupby('Symbol')['Adj Close'].pct_change()
merged_df['30D_Rolling_Volatility'] = (
    merged_df.groupby('Symbol')['Daily_Return'].rolling(window=30).std().reset_index(level=0, drop=True)
)
merged_df['90D_Rolling_Volatility'] = (
    merged_df.groupby('Symbol')['Daily_Return'].rolling(window=90).std().reset_index(level=0, drop=True)
)

# Step 4: Calculate Average Daily and Weekly Returns per Symbol
avg_daily_return = merged_df.groupby('Symbol')['Daily_Return'].mean()
avg_weekly_return = merged_df.groupby('Symbol')['Weekly_Return'].mean()

# Optional: Calculate other financial ratios if applicable, like P/E and Dividend Yield
# Ensure columns like 'EPS' and 'Dividend' are available in the DataFrame before computing these ratios
if 'EPS' in merged_df.columns:
    merged_df['P/E_Ratio'] = merged_df['Adj Close'] / merged_df['EPS']
if 'Dividend' in merged_df.columns:
    merged_df['Dividend_Yield'] = merged_df['Dividend'] / merged_df['Adj Close']

# Display the first few rows to check calculations
print(merged_df[['Symbol', 'Date', 'Adj Close', 'Weekly_Return', '30D_Rolling_Volatility', '90D_Rolling_Volatility']].head())


KeyError: 'Date'

In [ ]:
duplicates = merged_df[merged_df.index.duplicated()]
print(duplicates)

In [ ]:
# Replace the missing values with median values or Unknown where necessary 

#  Fill missing Ebitda values with the sector median
merged_df['Ebitda'] = merged_df.groupby('Sector')['Ebitda'].transform(lambda x: x.fillna(x.median()))

# Impute Revenuegrowth with sector median
merged_df['Revenuegrowth'] = merged_df.groupby('Sector')['Revenuegrowth'].transform(lambda x: x.fillna(x.median()))

# Fill missing states with a placeholder
merged_df['State'] = merged_df['State'].fillna("Unknown")

In [ ]:
# Convert object columns with limited unique values to 'category'
categorical_columns = ['Sector', 'Industry', 'Exchange', 'State', 'Country']
for col in categorical_columns:
    merged_df[col] = merged_df[col].astype('category')

# Check memory usage after conversion
print(merged_df.info())


- Feature Engineering

In [ ]:
# Calculate Daily Returns by calculating the percentage change in Adj Close for each stock over time.

merged_df['Daily_Return'] = merged_df.groupby('Symbol')['Adj Close'].pct_change()


merged_df['30D_Rolling_Volatility'] = merged_df.groupby('Symbol')['Daily_Return'].rolling(window=30).std().reset_index(0, drop=True)
# This line of code calculates the 30-day rolling volatility of daily returns for each symbol in the merged_df DataFrame
# In summary, this code computes a new column in merged_df that contains the 30-day rolling volatility (standard deviation) of daily returns for each symbol, allowing you to analyze how the volatility of each asset changes over time.



In [ ]:
merged_df.info()

In [ ]:
# Aggregate metrics by Sector with observed=True to silence the warning
sector_summary = merged_df.groupby('Sector', observed=True).agg(
    company_count=('Symbol', 'nunique'),
    total_market_cap=('Marketcap', 'sum'),
    avg_daily_return=('Daily_Return', 'mean'),
    volatility_daily_return=('Daily_Return', 'std'),
    max_price=('Currentprice', 'max'),
    min_price=('Currentprice', 'min'),
    total_volume=('Volume', 'sum')
).reset_index()

# Aggregate metrics by Industry with observed=True
industry_summary = merged_df.groupby('Industry', observed=True).agg(
    company_count=('Symbol', 'nunique'),
    total_market_cap=('Marketcap', 'sum'),
    avg_daily_return=('Daily_Return', 'mean'),
    volatility_daily_return=('Daily_Return', 'std'),
    max_price=('Currentprice', 'max'),
    min_price=('Currentprice', 'min'),
    total_volume=('Volume', 'sum')
).reset_index()


In [ ]:
# Export to CSV for Power BI
sector_summary.to_csv('sector_summary.csv', index=False)
industry_summary.to_csv('industry_summary.csv', index=False)

In [ ]:
merged_df.to_csv('cleaned_merged_stock_data.csv', index=False)


## Visualizing Stock Trends
- Plot stock price trends over time for selected companies to visualize performance.

In [ ]:
plt.figure(figsize=(12, 6))
for symbol in merged_df['Symbol'].unique()[:5]:  # Adjust for number of stocks to plot
    subset = merged_df[merged_df['Symbol'] == symbol]
    plt.plot(subset['Date'], subset['Adj Close'], label=symbol)
plt.title('Stock Price Trends for Selected Companies')
plt.xlabel('Date')
plt.ylabel('Adjusted Close Price')
plt.legend()
plt.grid(True)
plt.show()

## Performance Analysis:
- Analyze daily returns and visualize average returns against volatility.

In [ ]:
avg_returns = merged_df.groupby('Symbol')['Daily_Return'].mean()
volatility = merged_df.groupby('Symbol')['Daily_Return'].std()

plt.figure(figsize=(10, 6))
plt.scatter(volatility, avg_returns)
plt.title('Volatility vs Average Returns')
plt.xlabel('Volatility')
plt.ylabel('Average Daily Return')
plt.axhline(0, color='red', linestyle='--')
plt.axvline(0, color='red', linestyle='--')
plt.grid(True)
plt.show()

## Sector Performance:
- Visualize the performance of different sectors if sector data is included.

In [ ]:
# Calculate sector performance with observed=False to silence the warning
sector_performance = merged_df.groupby('Sector', observed=True)['Daily_Return'].mean().sort_values()
plt.figure(figsize=(10, 6))
sector_performance.plot(kind='barh')
plt.title('Average Daily Return by Sector')
plt.xlabel('Average Daily Return')
plt.ylabel('Sector')
plt.grid(True)
plt.show()

In [ ]:
plt.hist(merged_df['Daily_Return'].dropna(), bins=50, alpha=0.7)
plt.title('Histogram of Daily Returns')
plt.xlabel('Daily Return')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()